# Exp 001

In [1]:
import os
import shutil
import time
import pickle
import cv2 as cv
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime
from nets.sdnet import *
from ds.seq3 import *

np.set_printoptions(suppress=True)

In [2]:
def init_random_seed(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
init_random_seed(1234)

In [3]:
def set_gpu_memory_growth_mode(gpu_id=0):
    import tensorflow as tf
    # tf.config.gpu.set_per_process_memory_growth(True) --> TF 1.x
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if not gpus:
        return
    try:
        tf.config.experimental.set_memory_growth(gpus[gpu_id], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
set_gpu_memory_growth_mode()

In [4]:
def archive_weights(model):
    return [(v.name, v.numpy()) for v in model.trainable_variables]

In [5]:
def make_loss_fn():
    @tf.function
    def gaze_loss(label, pred):
        return tf.reduce_mean(tf.losses.mse(label, pred))

    @tf.function
    def cos_loss(label, pred):
        cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
        return cosine_loss(label, pred)
    
    @tf.function
    def euclidean_loss(label, pred):
        square = tf.math.square(pred - label)
        reduce_sum = tf.math.reduce_sum(square, axis=1)
        dists = tf.math.sqrt(reduce_sum)
        return tf.math.reduce_mean(dists)
    
    @tf.function
    def compound_loss(label, pred):
        pred_vec,  pred_xy  = tf.split(pred,  [3, 2], 1)
        label_vec, label_xy = tf.split(label, [3, 2], 1)
        xy_loss = gaze_loss(label_xy, pred_xy)
        vec_loss = gaze_loss(label_vec, pred_vec)
        xy_euc_loss = euclidean_loss(label_xy, pred_xy)
        return xy_loss + vec_loss + xy_euc_loss
    return compound_loss
 
def get_mean_distance_metric():
    def mean_distance(y_true, y_pred):
        _,  pred_xy = tf.split(y_pred, [3, 2], 1)
        _, label_xy = tf.split(y_true, [3, 2], 1)
        square = tf.math.square(pred_xy - label_xy)
        reduce_sum = tf.math.reduce_sum(square, axis=1)
        dists = tf.math.sqrt(reduce_sum)
        return tf.math.reduce_mean(dists)
    return mean_distance
    
def make_model():
    net = SDNet.create()
    l = make_loss_fn()
    o = tf.keras.optimizers.Adam(learning_rate=0.001)
    m = [get_mean_distance_metric()]
    net.compile(loss=l, optimizer=o, metrics=m)
    return net


# 데이터셋 

In [6]:
def load_dataset(gc_root_path, profiles, batch_size=32):
    data_bag = {}
    for pid in profiles:
        train = Sequence('train', pid, gc_root_path, batch_size=batch_size)
        valid = Sequence('valid', pid, gc_root_path, batch_size=batch_size)
        data_bag[pid] = { 't': train, 'v': valid }
    
    # sample shape show
    v = data_bag[profiles[0]]['v']
    inputs, targets = v[0]
    for ins in inputs:
        print(ins.shape)
    
    return data_bag

def cross_eval(m, data_bag, profiles):
    for pid in profiles:
        bag = data_bag[pid]
        t, v = bag['t'], bag['v']
        loss, metrics = m.evaluate(v, verbose=1, max_queue_size=10, workers=4)
        print("{} 평가".format(pid, loss, metrics))
        
def ce():
    cross_eval(model, data_bag, ps)

def train_epoch(m, data_bag, profile_id, snapshots, epochs=1, shuffle=False):
    bag = data_bag[profile_id]
    res = m.fit_generator(bag['t'], validation_data=bag['v'], epochs=epochs, verbose=1, 
                          shuffle=shuffle)
    w = archive_weights(model)
    s = (profile_id, w, res)
    snapshots.append(s)
    return res

def q(pid, epochs=1, at_once=False):
    if not at_once:
        for e in range(epochs):
            res = train_epoch(model, data_bag, pid, snapshots, epochs=1)
            print(">>> P[{}]: e[{}] snapshot: [{}]".format(pid, e, len(snapshots)-1))
    else:
        res = train_epoch(model, data_bag, pid, snapshots, epochs=epochs)
        

def compare(x, y):
    num_entry = len(snapshots)
    if x >= num_entry or y >= num_entry:
        print("only:", num_entry)
        return
    
    w_x, w_y = snapshots[x][1], snapshots[y][1]
    for i in range(0, len(w_x)):
        draw_weight_compare(w_x, w_y, i, (8,8))

In [7]:
snapshots = []
model = make_model()

In [8]:
# ps = ['01744', '02334', '02700', '01054', '01055', '02666']
ps = ['01050', '01051', '01054', '01055', '02666']
# ps = ['02342', '02349', '02450', '01905', '01054', '01055']
data_bag = load_dataset("/home/elvin/banner/mnt/ssd3/nps", ps)

(32, 64, 128, 1)
(32, 64, 128, 1)
(32, 9)
(32, 9)
(32,)
(32, 8)
(32, 3)


In [9]:
# data_bag['02666']['v'].visualize(500)

In [10]:
#stop

In [11]:
q(ps[1], 30, True)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 38 steps, validate for 20 steps
Epoch 1/30
38/38 [==============================] - 14s 373ms/step - loss: 30.9904 - mean_distance: 5.0348 - val_loss: 25.7747 - val_mean_distance: 4.1959
Epoch 2/30
38/38 [==============================] - 5s 123ms/step - loss: 17.9712 - mean_distance: 2.9657 - val_loss: 23.2543 - val_mean_distance: 3.7645
Epoch 3/30
38/38 [==============================] - 5s 123ms/step - loss: 13.4097 - mean_distance: 2.3710 - val_loss: 13.4263 - val_mean_distance: 2.3097
Epoch 4/30
38/38 [==============================] - 5s 124ms/step - loss: 11.3523 - mean_distance: 2.1352 - val_loss: 12.2031 - val_mean_distance: 2.3755
Epoch 5/30
38/38 [==============================] - 5s 123ms/step - loss: 10.1271 - mean_distance: 2.0108 - val_loss: 11.8866 - val_mean_distance: 2.4678
Epoch 6/30
38/38 [==============================] - 5s 123ms/

In [12]:
v = data_bag['01054']['v']
res = model.predict(v, verbose=2, max_queue_size=10, workers=4)

31/31 - 2s


In [13]:
ce()

  ...
    to  
  ['...']
22/22 [==============================] - 1s 28ms/step - loss: 14.0756 - mean_distance: 3.7177
01050 평가
  ...
    to  
  ['...']
20/20 [==============================] - 1s 27ms/step - loss: 5.4799 - mean_distance: 1.7967
01051 평가
  ...
    to  
  ['...']
31/31 [==============================] - 1s 26ms/step - loss: 12.0587 - mean_distance: 3.3572
01054 평가
  ...
    to  
  ['...']
29/29 [==============================] - 1s 26ms/step - loss: 12.3830 - mean_distance: 3.4859
01055 평가
  ...
    to  
  ['...']
24/24 [==============================] - 1s 26ms/step - loss: 8.3908 - mean_distance: 2.6378
02666 평가


In [14]:
compare(34, 35)

only: 1


# 평가

# 웨이트 변화량 실험

In [15]:
def cos_sim(x, y, eps=1e-08):
    norm_x = np.linalg.norm(x) + eps 
    norm_y = np.linalg.norm(y) + eps
    return np.dot(x, y) / (norm_x * norm_y)

In [16]:
def draw_weight_compare(w_x, w_y, idx, fs=(4,4)):
    # select layer
    wx_name, x = w_x[idx]
    wy_name, y = w_y[idx]
    
    if 'norm' in wx_name:
        return
    
    print(">>> name:{:20s} weight shape:".format(wx_name), x.shape)
    
    x, y = np.squeeze(x), np.squeeze(y)
    
    # weight reshape
    if len(x.shape) < 2:
        x = x.reshape(2, int(x.shape[-1]/2))
        y = y.reshape(2, int(y.shape[-1]/2))
    else:
        x = x.reshape(np.prod(x.shape[:-1]), x.shape[-1])
        y = y.reshape(np.prod(y.shape[:-1]), y.shape[-1])
    
    if np.prod(x.shape) < 8 and 'bias' not in wx_name:
        print("[w_x]:", x.T)
        print("[w_y]:", y.T)
        
    # full - weight similarity
    sim = np.around(cos_sim(x.flatten(), y.flatten()), decimals=4)
    
    # normalize
    avg_x, avg_y = np.mean(x), np.mean(y)
    min_x, max_x = np.min(x), np.max(x)
    min_y, max_y = np.min(y), np.max(y)
    
    x = (x - avg_x) / (max_x - min_x)
    y = (y - avg_y) / (max_y - min_y)
    x = x / np.linalg.norm(x)
    y = y / np.linalg.norm(y)
    
    # compute diff
    d = np.absolute(x - y)
    s = np.around(np.mean(d) * 1000, decimals=5)
    
    # if 'final' in wx_name:
    #     print(np.around(d, decimals=3))
    
    cm = 'hot'
    plt.figure(figsize=fs)
    plt.subplot(1, 3, 1)
    plt.imshow(x, cmap=plt.get_cmap(cm))
    plt.subplot(1, 3, 2)
    plt.imshow(y, cmap=plt.get_cmap(cm))
    plt.title('COS-Sim: {}'.format(sim))
    plt.subplot(1, 3, 3)
    plt.title('R: {}'.format(s))
    plt.imshow(d, cmap=plt.get_cmap(cm))

    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    
    plt.show()